# Assignment 1: Named Entity Recognition and Entity Linking Group Project

Authors:  
Alberto de los Ríos

## Imports

In [1]:
import pandas as pd
import os
import torch
import numpy as np
import huggingface_hub

In [13]:
# It is recommended to start with general import statements
#from utility_functions import *

## Load data

This section should load the raw dataset for the task.  
Remember to use relative paths to load any files in the notebook.

In [14]:
url_1177 = "hf://datasets/community-datasets/swedish_medical_ner/1177/train-00000-of-00001.parquet"
url_lt = "hf://datasets/community-datasets/swedish_medical_ner/lt/train-00000-of-00001.parquet"
url_wiki = "hf://datasets/community-datasets/swedish_medical_ner/wiki/train-00000-of-00001.parquet"

df_1177 = pd.read_parquet(url_1177)
df_lt = pd.read_parquet(url_lt)
df_wiki = pd.read_parquet(url_wiki)

os.makedirs("raw_data", exist_ok=True)
df_1177.to_parquet("raw_data/1177_train.parquet", engine="pyarrow")
df_lt.to_parquet("raw_data/lt_train.parquet", engine="pyarrow")
df_wiki.to_parquet("raw_data/wiki_train.parquet", engine="pyarrow")

## Task 1: LLMs for NER Survey

This survey will be evalutated based on the following metrics and dimensions:

- Language Capability (Swedish proficiency)
- Biomedical Knowledge (Domain relevance)
- Computational Efficiency (Training/inference costs)
- Performance Metrics (NER-specific scores)
- Ontology Compatibility (Linking to ICD/ICF/LOINC)

PONER UN POCO DE HISTORIA DE BERT creado por los researches the Google

For this first task we will only use the 1177 Vårdguiden dataset since it is the lightest with only 927 sentences.

# LLM in Swedish

Advantages:
1. Authenticity - Maintains original clinical nuance and terminology
2. No translation errors - Avoids introducing errors from machine translation
3. Proper noun handling - Swedish patient/place names and untranslatable terms remain intact
4. Future applicability - Model will work natively with Swedish EHR systems
5. Matching Swedish ontologies - Direct alignment with Swedish medical coding systems

Limitations:
1. Limited models - Fewer Swedish biomedical LLMs available:
- KB/bert-base-swedish-cased: [Swedish BERT models for NER](https://huggingface.co/KB/bert-base-swedish-cased)
- [RoBERTa large](https://huggingface.co/AI-Sweden-Models/roberta-large-1160k)
- Swedish GPT models with limited NER capacity: [AI Sweden Model Hub](https://huggingface.co/AI-Sweden-Models)
2. Smaller datasets - The Swedish medical NER dataset has only ~6,000 annotated entries
3. Debugging difficulty - Hard to verify annotations/errors without Swedish knowledge
4. Resource scarcity - Few Swedish stopword lists, tokenizers, etc.


# LLM in English

- [Swedish BERT models for NER](https://huggingface.co/KB/bert-base-swedish-cased)
- [RoBERTa large](https://huggingface.co/AI-Sweden-Models/roberta-large-1160k)
- [AI Sweden Model Hub](https://huggingface.co/AI-Sweden-Models)


Advantages:
1. Model availability - Access powerful English biomedical models:
- [BioBERT](https://github.com/naver/biobert-pretrained?tab=readme-ov-file)
- [ClinicalBERT](https://huggingface.co/medicalai/ClinicalBERT)
- [now BiomedBERT, previously known as PubMedBERT](https://huggingface.co/microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext)
2. Larger datasets - Can augment with English medical NER datasets (~20+ available)
3. Easier debugging - You can understand the text for error analysis
4. More tutorials - Abundant English NLP examples
5. Ontology linking - English ontologies (ICD-10 English) have more community support

Limitations:
1. Translation errors - Clinical terms often mistranslated:
2. Back-translation complexity - Need to map English predictions back to Swedish text
3. Loss of context - Swedish compound words get split unnaturally
4. Ontology mismatch - Swedish medical codes don't align perfectly with English
Added pipeline complexity - Requires translation component

- [BioBERT vs PubMedBERT](https://medium.com/@EleventhHourEnthusiast/model-comparison-biobert-vs-pubmedbert-8c2d78178d10)
- [Domain-Specific Language Model Pretraining for Biomedical Natural Language Processing](https://dl.acm.org/doi/10.1145/3458754)


# Multilingual LLM

- [mBERT](https://huggingface.co/google-bert/bert-base-multilingual-cased), 2018
- [XLM-RoBERTa](https://huggingface.co/FacebookAI/xlm-roberta-base), 2019


Advantages:
1. Cross-Lingual Knowledge Transfer: Leverages patterns from high-resource languages (e.g., German medical terms help Swedish). Fine-tune on Swedish data but benefit from pretraining on multilingual medical corpora.
2. Handling Code-Switching: No need for manual language detection.
3. Robust Tokenization: SentencePiece (used in XLM-R) handles Swedish compounds better than WordPiece (mBERT):
4. Future-Proofing: One model can support other languages (e.g., adding Norwegian EHRs later).

Limitations:
1. Translation errors - Clinical terms often mistranslated:
2. Back-translation complexity - Need to map English predictions back to Swedish text
3. Loss of context - Swedish compound words get split unnaturally
4. Ontology mismatch - Swedish medical codes don't align perfectly with English
Added pipeline complexity - Requires translation component

- [BioBERT vs PubMedBERT](https://medium.com/@EleventhHourEnthusiast/model-comparison-biobert-vs-pubmedbert-8c2d78178d10)
- [Domain-Specific Language Model Pretraining for Biomedical Natural Language Processing](https://dl.acm.org/doi/10.1145/3458754)


# Analyse raw data

In [15]:
#print("GPU available:", torch.cuda.is_available())

def analyse_df(df):

  df.columns
  print(f"Total number of sentences: {len(df)}")

  # Count sentences with at least one entity
  has_entity = df["entities"].apply(lambda ent: len(ent["start"]) > 0)
  print(f"Number of sentences with at least one entity: {has_entity.sum()}")

  from collections import Counter
  # Initialize counter
  type_counter = Counter()

  # Loop through entities and count each type
  for entity in df["entities"]:
      if "type" in entity:
          types = entity["type"]
          if isinstance(types, np.ndarray) and types.size > 0:
              type_counter.update(types.tolist())

  # Print results
  print("Entity counts from dataset:")
  print(f"Disorder/Finding (type 0): {type_counter[0]}")
  print(f"Pharmaceutical Drug (type 1): {type_counter[1]}")
  print(f"Body Structure (type 2): {type_counter[2]}")

#analyse_df(df_1177)
#analyse_df(df_lt)
analyse_df(df_wiki)



Total number of sentences: 48720
Number of sentences with at least one entity: 48720
Entity counts from dataset:
Disorder/Finding (type 0): 28387
Pharmaceutical Drug (type 1): 22858
Body Structure (type 2): 20592


 # Max number of Tokens

In [16]:
from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
tokenizer = AutoTokenizer.from_pretrained("KB/bert-base-swedish-cased")

def max_num_tokens(df):
    max_tokens = 0

    for sent in df["sentence"]:
        tokens = tokenizer(sent)["input_ids"]
        max_tokens = max(max_tokens, len(tokens))

    print("Maximum number of tokens:", max_tokens)

#max_num_tokens(df_1177)
#max_num_tokens(df_lt)
max_num_tokens(df_wiki)

Maximum number of tokens: 48


# Encoding and aligment of tokens and labels

In [17]:
from transformers import AutoTokenizer

# Load the tokenizer (multilingual, supports Swedish)
#tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
tokenizer = AutoTokenizer.from_pretrained("KB/bert-base-swedish-cased")

# BIO label mapping
label2id = {
    "O": 0,
    "B-DIS": 1,
    "I-DIS": 2,
    "B-DRUG": 3,
    "I-DRUG": 4,
    "B-BODY": 5,
    "I-BODY": 6
}

# Map integer entity types to BIO label prefixes
type_to_bio_prefix = {
    0: "DIS",
    1: "DRUG",
    2: "BODY"
}

def convert_to_bio_labels(encoding, entity_spans): #Converts character-level entity spans to BIO-formatted token-level labels.

    labels = [label2id["O"]] * len(encoding["input_ids"])

    for (ent_start, ent_end, ent_type) in entity_spans:
        bio_label = type_to_bio_prefix[ent_type]
        first_token = True

        for idx, (tok_start, tok_end) in enumerate(encoding["offset_mapping"]):
            if tok_start == tok_end == 0:
                continue  # Skip special tokens

            if tok_end > ent_start and tok_start < ent_end:  # Token overlaps entity
                if first_token:
                    labels[idx] = label2id[f"B-{bio_label}"]
                    first_token = False
                else:
                    labels[idx] = label2id[f"I-{bio_label}"]

    return labels

def process_dataset(df, num_examples):
    processed = []

    for i in range(len(df)):
        row = df.iloc[i]
        sentence = row["sentence"]
        entities = row["entities"]  # expects keys: 'start', 'end', 'type'

        encoding = tokenizer(
            sentence,
            return_offsets_mapping=True,
            truncation=True,
            padding="max_length",
            max_length=80 #max_number of tokens for df_1177=72 80 to be safe. check the other df
        )

        # Prepare entity spans for BIO conversion
        entity_spans = list(zip(entities["start"], entities["end"], entities["type"]))
        bio_labels = convert_to_bio_labels(encoding, entity_spans)
        encoding["labels"] = bio_labels

        # We no longer need offset_mapping
        encoding.pop("offset_mapping")

        if i < num_examples:
          print(f"\nSentence {i+1}:")
          tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
          for token, label_id in zip(tokens, encoding["labels"]):
              label_name = list(label2id.keys())[list(label2id.values()).index(label_id)]
              print(f"{token:15} -> {label_name}")

        processed.append(encoding)

    return processed

    #print("\nEncoding keys example:", encoding.keys())


#encodings_df_1177 = process_dataset(df_1177, 3)
#torch.save(encodings_df_1177, "processed_data/encodings_df_1177.pt") #Save the processed data
# Add this reverse mapping
id2label = {v: k for k, v in label2id.items()}
encodings_df_wiki = process_dataset(df_wiki, 3)
torch.save(encodings_df_wiki, "processed_data/encodings_df_wiki_se.pt") #Save the processed data



Sentence 1:
[CLS]           -> O
{               -> B-BODY
kropp           -> I-BODY
}               -> I-BODY
beskrivs        -> O
i               -> O
till            -> O
exempel         -> O
människok       -> O
##roppen        -> O
,               -> O
anat            -> O
##omi           -> O
och             -> O
f               -> O
[SEP]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD]           -> O
[PAD] 

In [18]:
#print(encodings_df_1177[0]["input_ids"]) #Print the keys of the first encoding
#print(encodings_df_1177[0]["attention_mask"])
#print(encodings_df_1177[0]["labels"])

Dataset is already tokenized and shows the spans intances and types of labels.


 What you do need to do:
1. Tokenize the sentence using a pretrained tokenizer, like BertTokenizerFast, which supports alignment of character offsets to token indices.

2. Use tokenizer(..., return_offsets_mapping=True) to map character positions (like 'start' and 'end') to token indices.

3. Create a labels list per sentence (same length as the number of tokens) initialized to -100 (special value that tells the model to ignore those positions), and then assign label IDs (0, 1, 2) to the tokens that overlap with entity spans.


CUENTA TAMBIEN LOS CARACTERES ESPECIALES ({[ DE CADA CLASE ASIQ VALORAR SI NO CONTARLOS O SI. PERO SON INTICATIVOS DE LA TYPO DE LABEL.

doubts:
1. Why do all sentences need to be the same length (via padding)?
Transformers process input as batches of fixed-size sequences. That’s because:

Matrix computations (done on GPU) require tensors of uniform shape.

Efficient batching boosts performance and stability during training.

2. Explanation of common fields in encoding:
- input_ids: The token IDs for your sentence (numbers representing tokens).
- token_type_ids: Used for distinguishing sentence segments (mostly for tasks like question answering). Might be all zeros if not applicable.
- attention_mask: Indicates which tokens are real (1) and which are padding (0).
- offset_mapping: For each token, the start and end character positions in the original sentence (useful for aligning labels).

3. Explicación:
- entities["type"] contains flat integers like 0, 1, 2, etc.
- type_to_bio_prefix maps those integers to entity type names (DIS, DRUG, etc.)
- convert_to_bio_labels() uses this mapping to apply B- and I- prefixes to the correct tokens.
- Then it converts those prefixes using label2id.


PONER EXPLICACIÓN DE BIO LABELS Y DE QUE MUCHOS MODELOS SOLO ACEPTAN ESTE TIPO DE LABELS.


# CREATE CUSTOM PYTORCH DATASET

Why create a Dataset class if you already have encodings?
- Integration with PyTorch DataLoader:
The Dataset class provides a standardized way for PyTorch to access your data. The DataLoader uses the Dataset to efficiently load data in batches, shuffle it, and handle multi-threaded loading.

- Batching & Shuffling:
When training a model, you typically don’t want to feed data one example at a time. You want mini-batches (e.g., 16 or 32 samples per batch). The DataLoader uses the Dataset to create batches, and you don’t have to manually slice your encodings.

- Lazy Access / Memory Efficiency:
The Dataset class lets PyTorch load one sample at a time on demand, rather than keeping everything as tensors in memory at once (especially useful with large datasets).

- Transforms / Augmentations:
If you want to apply any transformations (like token masking, noise, data augmentation) on the fly, the Dataset class is the place to do it — at retrieval time.

- Uniform Interface:
The Dataset abstracts the data representation so your training loop just works with the Dataset/DataLoader API without worrying about how data is stored.

- [Custom Named Entity Recognition with BERT](https://towardsdatascience.com/custom-named-entity-recognition-with-bert-cf1fd4510804/)


In [19]:
def collate_encodings(processed):
    encodings = {"input_ids": [], "attention_mask": []}
    labels = []
    for item in processed:
        encodings["input_ids"].append(item["input_ids"])
        encodings["attention_mask"].append(item["attention_mask"])
        labels.append(item["labels"])
    return encodings, labels

#encodings, labels = collate_encodings(encodings_df_1177)
encodings, labels = collate_encodings(encodings_df_wiki)

print(encodings.keys())
print(encodings["input_ids"][0])
print(encodings["attention_mask"][0])
print(labels[0])

dict_keys(['input_ids', 'attention_mask'])
[2, 20086, 2380, 17454, 11473, 31, 76, 1377, 38399, 14386, 19, 19041, 15664, 36, 13, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 5, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
import torch
from torch.utils.data import Dataset

class NERDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # For each key (input_ids, attention_mask), convert the sequence at idx to tensor
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Also convert labels for that idx to tensor
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Split and create the Dataset

In [21]:

from sklearn.model_selection import train_test_split

# Separate inputs and labels
all_input_ids = [enc['input_ids'] for enc in encodings_df_wiki]
all_attention_mask = [enc['attention_mask'] for enc in encodings_df_wiki]
all_labels = [enc['labels'] for enc in encodings_df_wiki]

# Pack inputs into one dictionary for convenience
all_encodings = {
    'input_ids': all_input_ids,
    'attention_mask': all_attention_mask
}

# Split

train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    all_input_ids, all_attention_mask, all_labels, test_size=0.2, random_state=42)

# Re-create encoding dicts for train and val
train_encodings = {
    'input_ids': train_input_ids,
    'attention_mask': train_attention_mask
}

val_encodings = {
    'input_ids': val_input_ids,
    'attention_mask': val_attention_mask
}

# Now create datasets
train_dataset = NERDataset(train_encodings, train_labels)
val_dataset = NERDataset(val_encodings, val_labels)





# Load Model

In [26]:
from transformers import XLMRobertaForTokenClassification, AutoModel, AutoTokenizer

# Number of labels (adjust to your label2id)
num_labels = len(label2id)

#model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base",num_labels=num_labels)

model = AutoModel.from_pretrained('KB/bert-base-swedish-cased', num_labels=num_labels)

c:\Users\beto1\OneDrive - KTH\1º\Applied Machine Learning and Artificial Inteligence\NER_Entity_Linking_Alberto_delosrios\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\beto1\.cache\huggingface\hub\models--KB--bert-base-swedish-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Training and Testing Setup

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./results",          # Where to save model/checkpoints
    evaluation_strategy="epoch",     # Evaluate after each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',            # TensorBoard logs
    logging_steps=10,
    save_strategy="epoch",           # Save checkpoint every epoch
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to=[]) # We'll define metrics below


from datasets import load_metric

metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    true_labels = [
        [list(label2id.keys())[list(label2id.values()).index(l)] for l in label_seq]
        for label_seq in labels
    ]
    true_predictions = [
        [list(label2id.keys())[list(label2id.values()).index(p)] for p in pred_seq]
        for pred_seq in predictions
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics)

trainer.train()

#trainer.save_model("models/fine_tuned_xlm_roberta_ner_wiki_se")
#tokenizer.save_pretrained("models/fine_tuned_xlm_roberta_ner_wiki_se")
trainer.save_model("models/fine_tuned_bert-base-swedish-cased_ner_wiki_se")
tokenizer.save_pretrained("models/fine_tuned_bert-base-swedish-cased_ner_wiki_se")
model.config.id2label = id2label
model.config.label2id = label2id
model.save_pretrained("models/fine_tuned_bert-base-swedish-cased_ner_wiki_se")



# Test Fine Tuned model with new sentences

In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

# 1. Load your fine-tuned model
#model_path = "models/fine_tuned_xlm_roberta_ner_wiki_se"  # adjust to your actual path
model_path = "models/fine_tuned_bert-base-swedish-cased_ner_wiki_se"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path, id2label=id2label, label2id=label2id)

# 2. Load NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


def predict_text(df):# 4. Loop through df_1177
    # 3. Prepare results list
    all_predictions = []

    for i, row in df.iterrows():
        text_id = row["sid"] if "sid" in df.columns else i  # fallback to index
        text = row["sentence"]  # adjust if your column has a different name

        predictions = ner_pipeline(text)
        for pred in predictions:
            all_predictions.append({
                "id": text_id,
                "text": text,
                "entity": pred["word"],
                "label": pred["entity_group"],
                "score": round(pred["score"], 3),
                "start": pred["start"],
                "end": pred["end"]
            })

    # 5. Convert to DataFrame
    results_df = pd.DataFrame(all_predictions)

    # 6. Save to CSV
    results_df.to_csv("results/ner_predictions_1177_se.csv", index=False, encoding="utf-8")

predict_text(df_1177)
print("Saved predictions to 'results/ner_predictions_1177_se.csv'")



In [2]:
# Load ICD-10-SE TSV file
icd_df = pd.read_csv('raw_data/icd-10-se.tsv', sep='\t', encoding='utf-8')

# Show the first few rows
icd_df.head()
#print(icd_df[['Code', 'Title', 'Description']].head())

column_renaming = {
    "Kod": "Code",
    "Giltig fr.o.m.": "Valid From",
    "Föräldrakod": "Parent Code",
    "Benämning": "Title",
    "Latin": "Latin",
    "Beskrivning": "Description",
    "Exempel": "Example",
    "Innefattar": "Includes",
    "Innefattar ej": "Excludes",
    "Anmärkning": "Note",
    "Kodningsinformation": "Coding Information",
    "Innehåll": "Content",
    "Manifestation (*)/ Etiologi (†)": "Manifestation (*)/ Etiology (†)",
    "Länkad manifestation (*)/ etiologi (†)": "Linked Manifestation (*)/ Etiology (†)",
    "Inte huvuddiagnos": "Not Primary Diagnosis",
    "Kodnivå - kodspecifikation": "Code Level - Code Specification"
}





C:\Users\beto1\AppData\Local\Temp\ipykernel_15252\4280058965.py:2: DtypeWarning: Columns (4,5,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  icd_df = pd.read_csv('raw_data/icd-10-se.tsv', sep='\t', encoding='utf-8')


## Task 2: This is the title of task 2

This section should contain the solution of task 2.

## Results and Discussion

This section should contain:
- Results.
- Summary of best model performance:
    - Name of best model file as saved in /models.
    - Relevant scores such as: accuracy, precision, recall, F1-score, etc.
- Key discussion points.

In [ ]:
# Always use comments in the code to document specific steps